In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM
from typing import Tuple, List, Optional
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from langchain.chains import GraphCypherQAChain

from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph

import torch

import pickle
import os

In [2]:
NEO4J_URI="neo4j+s://9269fc71.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="lOnpSSiAFQg0uWzA1EvYYBhK45_rWbxXb9SHRDwZRfk"

In [3]:
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [4]:
graph = Neo4jGraph()

In [5]:

# Load the cleaned_documents list from the file
with open('final_pdf_pages.pkl', 'rb') as file:
    docs = pickle.load(file)

print("Documents loaded successfully!")


Documents loaded successfully!


In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separators=['\n'],
    chunk_size=2024,
    chunk_overlap=506,
    length_function=len,
)

In [7]:
data = text_splitter.split_documents(docs)

In [8]:
type(data[0])

langchain_core.documents.base.Document

In [9]:
len(data)

667

In [13]:
torch.cuda.empty_cache()

In [14]:
!nvidia-smi

Mon Nov  4 19:27:06 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:19:00.0 Off |                  Off |
| 30%   29C    P5             11W /  230W |      24MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
llm = OllamaLLM(model="llama3.1:70b", num_gpu=torch.cuda.device_count())

In [16]:
# Define allowed nodes and relationships based on the medical context
allowed_nodes = ["Disease", "Cause", "Symptom", "Complication", "Treatment", "Medication", "Test", "Risk Factor", "Diagnosis", "Side Effect", "Procedure", "Condition Stage", "Disease Type", "Imaging Type"]
allowed_relationships = ["CAUSES", "HAS_SYMPTOM", "COMPLICATES", "TREATED_WITH", "DIAGNOSED_BY", "HAS_RISK_FACTOR", "INDICATES", "ASSOCIATED_WITH", "CONTRADINDICATED_WITH"]

In [17]:
graph_transformer = transformer = LLMGraphTransformer(
                                    llm=llm,
                                    allowed_nodes=allowed_nodes,
                                    allowed_relationships=allowed_relationships,
                                    node_properties=False, 
                                    relationship_properties=False,
                                    # prompt=graph_conversion_template
                                )

In [ ]:
graph_documents = graph_transformer.convert_to_graph_documents(data)

In [ ]:
with open('graph_documents.pkl', 'wb') as file:
    pickle.dump(graph_documents, file)

print("Documents saved successfully!")

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)